In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score  #交叉验证评分
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error #均方根误差
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv(r'F:\TItanic\data\train_chuli.csv')

In [2]:
# 输出数据预览
print(dataset.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  \
0            1         0       3    1  22.0      1      0   7.2500   
1            2         1       1    0  38.0      1      0  71.2833   
2            3         1       3    0  26.0      0      0   7.9250   
3            4         1       1    0  35.0      1      0  53.1000   
4            5         0       3    1  35.0      0      0   8.0500   

   family_size  Embarked_C  Embarked_Q  Embarked_S  
0            2           0           0           1  
1            2           1           0           0  
2            1           0           0           1  
3            2           0           0           1  
4            1           0           0           1  


In [3]:
from sklearn.model_selection import train_test_split #划分数据集
# 准备训练数据
x = dataset.drop(['Survived'], axis = 1)
y = dataset['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(623, 11) (268, 11) (623,) (268,)


In [4]:
#评价指标
import numpy as np

# 计算混淆矩阵
def compute_confusion_matrix(precited,expected):
    part = precited ^ expected             # 对结果进行分类，亦或使得判断正确的为0,判断错误的为1
    pcount = np.bincount(part)             # 分类结果统计，pcount[0]为0的个数，pcount[1]为1的个数
    tp_list = list(precited & expected)    # 将TP的计算结果转换为list
    fp_list = list(precited & ~expected)   # 将FP的计算结果转换为list
    tp = tp_list.count(1)                  # 统计TP的个数
    fp = fp_list.count(1)                  # 统计FP的个数
    tn = pcount[0] - tp                    # 统计TN的个数
    fn = pcount[1] - fp                    # 统计FN的个数
    return tp, fp, tn, fn

# 计算常用指标
def compute_indexes(tp, fp, tn, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)     # 准确率
    precision = tp / (tp+fp)               # 精确率
    recall = tp / (tp+fn)                  # 召回率
    F1 = (2*precision*recall) / (precision+recall)    # F1
    return accuracy, precision, recall, F1


In [5]:
# !pip install pygam

In [6]:
from matplotlib.cm import rainbow 
from pygam import LogisticGAM
#决策树
rf_scores = []

rf_classifier = LogisticGAM()
rf_classifier.fit(x_train, y_train)

LogisticGAM(callbacks=[Deviance(), Diffs(), Accuracy()], 
   fit_intercept=True, max_iter=100, 
   terms=s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10) + intercept,
   tol=0.0001, verbose=False)

LogisticGAM
- Intercept（截距）：在所有输入变量都为零时的输出值，即模型的基本预测。
- Spline functions（样条函数）：在 LogisticGAM 中，样条函数用于拟合变量与目标之间的非线性关系。样条函数通常使用自由度（degree of freedom，df）参数控制其光滑度。df 参数越大，函数就越光滑，但同时也更容易出现过拟合。
- Linear terms（线性项）：如果在模型中指定了线性项，则它们将被视为与目标变量之间的线性关系。
- Interaction terms（交互项）：交互项表示两个变量之间的交互作用。例如，如果一个模型包含“性别”和“年龄”两个变量，它们的交互项将考虑性别和年龄的组合如何影响输出变量。
- Penalty parameters（惩罚参数）：LogisticGAM 通常使用惩罚项来控制模型的复杂度。这些惩罚项通常使用惩罚参数控制，通常用于帮助减少过拟合。
- Link function（联系函数）：LogisticGAM 通常使用逻辑斯蒂函数作为链接函数，将模型输出限制在 [0,1] 范围内。但是，可以使用其他链接函数，具体取决于模型的目标和数据类型。

In [7]:
rfc_score = rf_classifier.accuracy(x_test, y_test)#测试集准确率,j即accuracy
expected = y_test.astype('int32')#预期值
# print(type(expected))
# print(expected)
rfc_pre=rf_classifier.predict(x_test)
precited = rfc_pre.astype('int32')#模型预测值
# print(type(precited))
tp, fp, tn, fn = compute_confusion_matrix(precited, expected)
accuracy, precision, recall, F1 = compute_indexes(tp, fp, tn, fn)
rfc_pre = rf_classifier.predict(x_test)
rfc_auc = metrics.roc_auc_score(y_test, rfc_pre)
rfc_rmse = np.sqrt(mean_squared_error(y_test, rfc_pre))
print('TP FP TN FN')
print('%d %d %d %d'%(tp,fp,tn,fn))
print('accuracy precision recall F1')
print('%.4lf %.4lf %.4lf %.4lf'%(accuracy, precision, recall, F1))
print('auc rmse')
print('%.4lf %.4lf'%(rfc_auc, rfc_rmse))

TP FP TN FN
71 27 141 29
accuracy precision recall F1
0.7910 0.7245 0.7100 0.7172
auc rmse
0.7746 0.4571
